<a href="https://colab.research.google.com/github/rromerov/Machine-Learning-Projects/blob/main/Credit_Card_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split

Load the data and replace the column names from German to English as indicated on the UCI page. 

In [28]:
file="https://raw.githubusercontent.com/rromerov/Machine-Learning-Projects/main/SouthGermanCredit.asc"
credit=pd.read_table(file,sep=" ")

In [29]:
credit.head()

,laufkont,laufzeit,moral,verw,hoehe,sparkont,beszeit,rate,famges,buerge,...,verm,alter,weitkred,wohn,bishkred,beruf,pers,telef,gastarb,kredit
0,1,18,4,2,1049,1,2,4,2,1,...,2,21,3,1,1,3,2,1,2,1
1,1,9,4,0,2799,1,3,2,3,1,...,1,36,3,1,2,3,1,1,2,1
2,2,12,2,9,841,2,4,2,2,1,...,1,23,3,1,1,2,2,1,2,1
3,1,12,4,0,2122,1,3,3,3,1,...,1,39,3,1,2,2,1,1,1,1
4,1,12,4,0,2171,1,3,4,3,1,...,2,38,1,2,2,2,2,1,1,1


In [48]:
columns_name=["status","duration","credit_history","purpose","amount","savings","employment_duration","installment_rate","personalstatussex","other_debtors","president_residence","property","age","otherinstallmentplans","housing","number_credits","job","people_liable","telephone","foreign_worker","target column"]
credit.columns=columns_name

In [53]:
for i in credit.columns:
  unique_values_per_column=credit[i].unique()
  sorted_values=sorted(unique_values_per_column)
  print(i, sorted_values)

status [1, 2, 3, 4]
duration [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 20, 21, 22, 24, 26, 27, 28, 30, 33, 36, 39, 40, 42, 45, 47, 48, 54, 60, 72]
credit_history [0, 1, 2, 3, 4]
purpose [0, 1, 2, 3, 4, 5, 6, 8, 9, 10]
amount [250, 276, 338, 339, 343, 362, 368, 385, 392, 409, 426, 428, 433, 448, 454, 458, 484, 518, 522, 571, 585, 590, 601, 609, 618, 625, 626, 629, 639, 640, 652, 654, 660, 662, 666, 672, 674, 682, 683, 684, 685, 691, 697, 700, 701, 707, 708, 709, 713, 717, 719, 727, 730, 741, 745, 750, 753, 754, 759, 760, 763, 766, 776, 781, 783, 790, 795, 797, 802, 804, 806, 836, 841, 846, 860, 866, 874, 882, 884, 886, 888, 894, 900, 902, 907, 909, 915, 918, 926, 929, 930, 931, 932, 935, 936, 937, 939, 947, 950, 951, 958, 959, 960, 975, 976, 983, 996, 999, 1007, 1024, 1028, 1037, 1038, 1042, 1047, 1048, 1049, 1050, 1053, 1055, 1056, 1068, 1076, 1082, 1092, 1098, 1101, 1103, 1107, 1108, 1113, 1123, 1126, 1131, 1136, 1138, 1149, 1154, 1155, 1158, 1163, 1164, 1168, 1169, 1175, 118

In [5]:
credit.head()

,status,duration,credit_history,purpose,amount,savings,employment_duration,installment_rate,personalstatussex,other_debtors,...,property,age,otherinstallmentplans,housing,number_credits,job,people_liable,telephone,foreign_worker,target column
0,1,18,4,2,1049,1,2,4,2,1,...,2,21,3,1,1,3,2,1,2,1
1,1,9,4,0,2799,1,3,2,3,1,...,1,36,3,1,2,3,1,1,2,1
2,2,12,2,9,841,2,4,2,2,1,...,1,23,3,1,1,2,2,1,2,1
3,1,12,4,0,2122,1,3,3,3,1,...,1,39,3,1,2,2,1,1,1,1
4,1,12,4,0,2171,1,3,4,3,1,...,2,38,1,2,2,2,2,1,1,1


Partition the data into the 85% training set and the 15% test set. The models will be trained with the cross-validation method, so it is not necessary at this step to generate the validation set. Define as the X variable all input variables and the Y variable as the output variable. 

In [19]:
X=credit.iloc[:,0:-2]
y=credit.iloc[:,-1]

In [20]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.15,random_state=1)